In [1]:
# Import the modules
import pandas as pd
from pathlib import Path
import hvplot.pandas
from sklearn.cluster import KMeans
from pathlib import Path
import requests

## Load the Credit Card Data into a Pandas DataFrame

In [2]:
# Read in the CSV file as a Pandas DataFrame
movies_cleaned_df = pd.read_csv(
    Path("Resources/movies_cleaned.csv")
)

# Review the DataFrame
movies_cleaned_df.head()

,title,extracted_genres
0,Toy Story,Animation
1,Jumanji,Adventure
2,Grumpier Old Men,Romance
3,Waiting to Exhale,Comedy
4,Father of the Bride Part II,Comedy


## Transform "extracted_genres" column with get_dummies

In [3]:
# Verify the categories of the "extracted_genres" column
movies_cleaned_df["extracted_genres"].value_counts()

Drama              11804
Comedy              8517
Action              4464
Documentary         3376
Horror              2612
Crime               1663
Thriller            1646
Adventure           1502
Romance             1163
Animation           1113
Fantasy              696
Science Fiction      639
Mystery              547
Family               508
Music                480
Western              449
TV Movie             380
War                  373
History              273
Foreign              116
Name: extracted_genres, dtype: int64

In [4]:
# Transform the extracted_genres column using get_dummies
genres_dummies = pd.get_dummies(movies_cleaned_df["extracted_genres"])

# Display the transformed data
genres_dummies.tail()

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,Foreign,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
42316,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
42317,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
42318,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
42319,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
42320,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
# Concatenate the movies_cleaned_df and the genres_dummies DataFrames
#genresinfo_df = pd.concat([movies_cleaned_df, genres_dummies], axis=1)

# Drop the original education column
#genresinfo = genres_dummies.drop(columns=["movies_cleaned_df","extracted_genres"])
genresinfo=genres_dummies
# Display the DataFrame
genresinfo.head()

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,Foreign,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## PCA to reduce dimensionality

In [6]:
# Import the PCA module
from sklearn.decomposition import PCA

In [7]:
# Instantiate the PCA instance and declare the number of PCA variables
pca=PCA(n_components=2)

In [8]:
# Fit the PCA model on the transformed credit card DataFrame
genres_pca = pca.fit_transform(genresinfo)

# Review the first 5 rows of the array of list data
genres_pca[:5]

array([[-0.12231177,  0.22569747],
       [-0.12773667,  0.243542  ],
       [-0.12298383,  0.22784223],
       [-0.63801794, -0.57191662],
       [-0.63801794, -0.57191662]])

### Step 2: Using the explained_variance_ratio_ function from PCA, calculate the percentage of the total variance that is captured by the two PCA variables.

In [9]:
# Calculate the PCA explained variance ratio
pca.explained_variance_ratio_

array([0.28444457, 0.17781322])

In [10]:
# Create the PCA DataFrame
genres_pca_df = pd.DataFrame(
    genres_pca,
    columns=["PCA1", "PCA2"]
)

# Review the PCA DataFrame
genres_pca_df.head()

,PCA1,PCA2
0,-0.122312,0.225697
1,-0.127737,0.243542
2,-0.122984,0.227842
3,-0.638018,-0.571917
4,-0.638018,-0.571917


## Elbow Method to find k

In [11]:
# Import the KMeans module from SKLearn
from sklearn.cluster import KMeans

In [12]:
# Create a a list to store inertia values and the values of k
inertia = []
k = list(range(1, 11))

In [13]:
# Create a for-loop where each value of k is evaluated using the K-means algorithm
# Fit the model using the service_ratings DataFrame
# Append the value of the computed inertia from the `inertia_` attribute of the KMeans model instance
for i in k:
    k_model = KMeans(n_clusters=i, random_state=0)
    k_model.fit(genres_pca_df)
    inertia.append(k_model.inertia_)

/Users/angaddeepdhillon/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/angaddeepdhillon/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/angaddeepdhillon/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/angaddeepdhillon/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/cluster/

In [14]:
# Define a DataFrame to hold the values for k and the corresponding inertia
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)

# Review the DataFrame
df_elbow.head()

,k,inertia
0,1,16694.958463
1,2,7116.738868
2,3,512.648214
3,4,88.505705
4,5,15.179914


In [15]:
# Plot the DataFrame
df_elbow.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k
)

:Curve   [k]   (inertia)

## ## Kmeans algo to cluster data

In [16]:
# Define the model with 3 clusters
model = KMeans(n_clusters=3, random_state=3)

# Fit the model
model.fit(genres_pca_df)

# Make predictions
k_3 = model.predict(genres_pca_df)

# Create a copy of the preprocessed data
genresinfo_predictions_df = genres_pca_df.copy()

# Add a class column with the labels
genresinfo_predictions_df['genres_segments'] = k_3

/Users/angaddeepdhillon/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [17]:
# Plot the clusters
genresinfo_predictions_df.hvplot.scatter(
    x="PCA1",
    y="PCA2",
    by="genres_segments"
)

:NdOverlay   [genres_segments]
   :Scatter   [PCA1]   (PCA2)